# Demo: Stain Normalization

<a href="https://colab.research.google.com/github/TIA-Lab/tiatoolbox/blob/doc-stainnorm/examples/example_stainnorm.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://kaggle.com/kernels/welcome?src=https://github.com/TIA-Lab/tiatoolbox/blob/doc-stainnorm/examples/example_stainnorm.ipynb" target="_blank"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/></a>

## Prerequisites

## About this notebook
This jupyter notebook can be run on any computer with a standard browser and no programming language is required. It can run remotely over the Internet, free of charge, thanks to Google Colaboratory or Kaggle. (The Kaggle version is temporarily unavailable.) To connect with Colab or Kaggle, click on one of the two blue checkboxes above. Check that "colab" appears in the browser address bar after you click on "Open in Colab". Familiarize yourself with the drop-down menus near the top of the window. You can edit the notebook during the session, for example substituting your own image files for the image files used in this demo. Experiment by changing the parameters of functions. It is not possible for an ordinary user to permanently change this version of the notebook on either Github or colab, so you cannot inadvertently mess it up. Use the notebook's File Menu if you wish to save your own (changed) notebook.

Before running the notebook outside Colab, set up your Python environment, as explained in the 
[README](https://github.com/TIA-Lab/tiatoolbox/blob/develop/README.md#install-python-package) file.
### Welcome to tiatoolbox!

This is a demo of normalisation of stain coloring in a whole slide image (WSI). We will
1. Load a sample WSI.
2. Extract a tile.
3. Stain-normalise the tile with various built-in methods.
4. Stain-normalise with a user-defined stain matrix.<br/>

During the above steps, we will be using functions from our `stainnorm` module [here](https://github.com/TIA-Lab/tiatoolbox/blob/master/tiatoolbox/tools/stainnorm.py). This demo assumes some understanding of the functions in the `wsireader` module (for example by going through the demo [here](https://github.com/TIA-Lab/tiatoolbox/blob/master/examples/example_wsiread.ipynb)).

### First cell in bash
The first line of code ssys that the rest of the cell will be interpreted in bash. The second line removes the directory `tmp` if it exists–a previous run may have created it. The other lines set up a suitable Python environment under Colab. No output is expected if the notebook is either run outside Colab or run for a second time in the same Colab session. 

In [ ]:
%%bash
[ -d tmp ] && ( echo "deleting tmp directory"; rm -rf tmp )
if [ $(env | grep COLAB | wc -c) -gt 0 ] && [ $( pip list | grep datascience | wc -c) -gt 0 ]
then
    echo "uninstalling colab packages with conflicts"
    pip uninstall -y datascience
    pip uninstall -y albumentations
    apt-get -y install libopenjp2-7-dev libopenjp2-tools openslide-tools
    pip install tiatoolbox
fi

We start by importing some necessary modules.

In [ ]:
import cv2
from tiatoolbox.dataloader.slide_info import slide_info
from tiatoolbox.dataloader import wsireader
import requests
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 150 # for high resolution figure in notebook

## Reading in a WSI

We load a small WSI, specified with the help of the path variable `user_sample_wsi_path`(default value `None`). Unless this None value is changed, the WSI is downloaded from the web, as seen in the code below, and saved with filename given by the variable `sample_file_name` in the directory given by `data_dir`. Data generated by the notebook is stored under data_dir, providing rapid local access..


In [ ]:
data_dir = './tmp'
sample_file_name = 'sample_wsi_small.svs'

user_sample_wsi_path = None

def download(url_path, save_path):
    r = requests.get(url_path)
    with open(save_path, "wb") as f:
        f.write(r.content)
    return

user_sample_wsi_path = None

if user_sample_wsi_path is None:
    sample_wsi_path = '%s/%s' % (data_dir, sample_file_name)
else:
    sample_wsi_path = user_sample_wsi_path
if not os.path.exists(sample_wsi_path):
    os.makedirs(os.path.dirname(sample_wsi_path))
    url_path = "http://openslide.cs.cmu.edu/download/openslide-testdata/Aperio/CMU-1-Small-Region.svs"
    download(url_path, sample_wsi_path)

We read the WSI sample, and print important metadata. We plot a thumbnail version to easily find regions of interest.


In [ ]:
# create a file handler
wsi_reader = wsireader.get_wsireader(
                input_img=sample_wsi_path)
wsi_info = wsi_reader.info.as_dict()
# we will print out each info line by line
print(*list(wsi_info.items()), sep='\n')
wsi_thumb = wsi_reader.slide_thumbnail(resolution=1.25, units='power')

plt.imshow(wsi_thumb)
plt.show()


## Extracting a tile from the WSI

From the figure, a `50x50` tile centered at position `[75, 125]` (XY-coordinate) in the thumbnail contains both the stroma and the gland tissue component. This would be a nice sample for different staining method. The thumbnail is loaded at x1.25 objective power. We proceed to convert the position and size to `[800, 1600]` and `800x800` at x16 objective power and load the area up.

In [ ]:
sample = wsi_reader.read_region(
            location=[800, 1600], # in X, Y
            level=0, size=[800, 800])
plt.imshow(sample)
plt.axis('off')
plt.show()


## Stain normalising a tile

Now we will download a different image to act as target stain.

In [ ]:
# downloading the target image
url_path = "https://raw.githubusercontent.com/TIA-Lab/tiatoolbox/develop/data/target_image.png"
target_image_path = '%s/stain_target.png' % (data_dir)
download(url_path, target_image_path)

target_image = cv2.imread(target_image_path)
target_image = cv2.cvtColor(target_image, cv2.COLOR_BGR2RGB)
plt.imshow(target_image)
plt.axis('off')
plt.show()

### Vahadane method

We can stain-normalise the `sample` image to the target image using the widely known [Vahadane](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7460968) method. The normaliser follow the `sklearn` calling style and is applied as follows:

In [ ]:
from tiatoolbox.tools import stainnorm

stain_normalizer = stainnorm.VahadaneNormaliser()
stain_normalizer.fit(target_image)

normed_sample = stain_normalizer.transform(sample.copy())

plt.figure(figsize=(10, 5))
plt.subplot(1,3,1)
plt.imshow(target_image)
plt.title('Target Image')
plt.axis('off')
plt.subplot(1,3,2)
plt.imshow(sample)
plt.title('Source Image')
plt.axis('off')
plt.subplot(1,3,3)
plt.imshow(normed_sample)
plt.title('Vahadane Stain Normed')
plt.axis('off')
plt.show()


### Other stain normalisation methods

At the moment, the library contains implementation for 4 stain-normalisation methods. You can either create them by using their class name like above, or our provided getter function `get_normaliser` by inputting the corresponding method name. Here, we will illustrate how you can use the latter approach to get the normaliser.

The stain normalisation names are provide in `method_name_list` variable below. We will sequentially apply each method on the `sample` image and plot them up for visual comparison.

In [ ]:
method_name_list = ['reinhard', 'ruifrok', 'macenko', 'vahadane']

plt.subplot(2,3,1)
plt.imshow(sample)
plt.title('Source Image')
plt.axis('off')
plt.subplot(2,3,4)
plt.imshow(target_image)
plt.title('Target Image')
plt.axis('off')

pos = [2, 3, 5, 6]
for idx, method_name in enumerate(method_name_list):
    stain_normalizer = stainnorm.get_normaliser(method_name)
    stain_normalizer.fit(target_image)

    normed_sample = stain_normalizer.transform(sample.copy())
    plt.subplot(2,3,pos[idx])
    plt.imshow(normed_sample)
    plt.title(method_name.capitalize())
    plt.axis('off')
plt.tight_layout()
plt.show()

## Custom staining

In addition to the 4 implemented methods, you can also extend the basic functionalities to implement your own method. One way is to subclass the base class `stainnorm.StainNormaliser` and implement your method. Another way is to provide your custom stain conversion matrix and input it to the `stainnorm.StainNormaliser`. Note, the former is for advanced users only! In this notebook, we will only explore the customisation of the input stain matrix. As an example, we will use the stain matrix in `skimage` library for getting Feulgen + Light Green stained from RGB. We will finally compare the stain-normalisation result using a custom defined stain matrix and using the Vahadane method.

In [ ]:

import skimage.color

stain_matrix = skimage.color.fgx_from_rgb[:2]
custom_normalizer = stainnorm.CustomNormaliser(stain_matrix)
custom_normalizer.fit(target_image)

vahadane_normalizer = stainnorm.VahadaneNormaliser()
vahadane_normalizer.fit(target_image)

normed_sample1 = custom_normalizer.transform(sample.copy())
normed_sample2 = stain_normalizer.transform(sample.copy())

plt.subplot(2,2,1)
plt.imshow(sample)
plt.title('Source Image')
plt.axis('off')
plt.subplot(2,2,3)
plt.imshow(target_image)
plt.title('Target Image')
plt.axis('off')
plt.subplot(2,2,2)
plt.imshow(normed_sample1)
plt.title('Custom Stain Matrix')
plt.axis('off')
plt.subplot(2,2,4)
plt.imshow(normed_sample2)
plt.title('Vahadane')
plt.axis('off')
plt.show()